# Dependencies Installation

In [1]:
# !pip cache dir
!pip cache purge
!pip install --upgrade pip
!pip install twilio flask requests flask-ngrok
!pip install -q transformers
!pip install joblib
!pip install tensorflow
!pip install torch torchvision
!pip install wheel
!pip install pandas
!pip install unzip

Files removed: 1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!killall ngrok

ngrok: no process found


In [3]:
# to restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# Imports

In [4]:
from transformers import *
import tensorflow as tf;
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import urllib.request
import json
import pandas as pd

from flask import Flask, request
import requests
from twilio.twiml.messaging_response import Message, MessagingResponse
from flask_ngrok import run_with_ngrok

# Using - Model
https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es

In [5]:
nlp = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False}
    )
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mrm8488--distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es/snapshots/a8842522e00a16b382c875ecb2da2dd8cf7cf5b6/config.json
Model config BertConfig {
  "_name_or_path": "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

loading configuration file config.json from cache at /root/.cache/hugg

In [6]:
context_test = 'Isaac Peral fue un murciano que inventó el submarino'
question_test = '¿Quién inventó el submarino?'

In [7]:
nlp(
    {
        'question': question_test,
        'context': context_test
                    
    }
)

{'score': 0.9515752792358398, 'start': 0, 'end': 11, 'answer': 'Isaac Peral'}

# Answer questions based on a context

In [8]:
def contextQuestionResponse(context, question):
  data = {
        'context': context,
        'question': question
  }

  body = str.encode(json.dumps(data))

  try:
    response = nlp(data)
    if data != "":
      #print(response)
      return response["answer"]
    else:
      #print(response)
      return "La solicitud falló."
  except response as error:
    print("The request failed: " + str(error))

In [9]:
# Testing function
contextQuestionResponse(context_test,question_test)

'Isaac Peral'

# Authenticating in Ngrok

In [10]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#Sign in with oscar romero
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

#If neccesary, replace this authtoken

#Commented for security.
!/ngrok authtoken 2F0ojhoOQFrC9shlmrTZ3RA3dl9_4CyAwVMe79EpJUG2HArfY

/drive/ngrok-ssh
--2022-10-22 03:17:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  5.38MB/s    in 2.5s    

2022-10-22 03:17:30 (5.38 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Launching

In [12]:
class Context:

  def __init__(self, context = ''):
         self._context = context
    
  def get_context(self):
        return self._context
      
  def set_context(self, c):
        self._context = c

In [13]:
context = Context()
app = Flask(__name__)
run_with_ngrok(app)

#https://api.whatsapp.com/send/?phone=14155238886

@app.route('/bot', methods=['POST'])

def bot():
    
    incoming_msg = request.values.get('Body', '').lower()
    print("\n===========================")
    print("MENSAJE RECIBIDO: ", incoming_msg)
    print("===========================")
    response  = MessagingResponse()
    msg = response.message()
    print(context.get_context())
    responded = False   

    if 'hola' in incoming_msg:
        msg.media('https://t3.ftcdn.net/jpg/02/23/56/34/360_F_223563498_k5Gedp6VMuZ2zwCpbNqIoQFuMXHH2JUc.jpg')  
        msg_resp = 'Bienvenido! por favor ingresa un texto sobre el que quieras hacer una pregunta.'
        msg.body(msg_resp)
        responded = True

    if 'texto' in incoming_msg:
        context.set_context(incoming_msg)
        msg_resp = 'Ingresa una pregunta por favor y me vere en la obligación de responder con completa sinceridad.'
        msg.body(msg_resp)
        responded = True

    if '?' in incoming_msg:
        msg_resp = contextQuestionResponse(context.get_context(), incoming_msg)
        msg.body(msg_resp)
        responded = True 
    
    if not responded:
        msg.body('Lo siento, sólo se de textos y de preguntas.')
    return str(response)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4636-146-148-63-188.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Oct/2022 03:26:40] "POST /bot HTTP/1.1" 200 -



MENSAJE RECIBIDO:  hola



INFO:werkzeug:127.0.0.1 - - [22/Oct/2022 03:27:06] "POST /bot HTTP/1.1" 200 -



MENSAJE RECIBIDO:  texto: esta semana también se conoció que trump compartió cartas clasificadas de kim jong-un, el líder norcoreano, con el periodista bob woodward. al parecer estos documentos se encontraban entre la información que los archivos nacionales recuperaron de su casa en florida. en los documentos que se recuperaron en enero de este año, también estaba una carta que el expresidente barack obama le dejó a trump. esto hace parte de una tradición en la que el presidente saliente deja una misiva para su sucesor


MENSAJE RECIBIDO:  ¿qué hizo trump?
texto: esta semana también se conoció que trump compartió cartas clasificadas de kim jong-un, el líder norcoreano, con el periodista bob woodward. al parecer estos documentos se encontraban entre la información que los archivos nacionales recuperaron de su casa en florida. en los documentos que se recuperaron en enero de este año, también estaba una carta que el expresidente barack obama le dejó a trump. esto hace parte de una tradi

INFO:werkzeug:127.0.0.1 - - [22/Oct/2022 03:27:29] "POST /bot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Oct/2022 03:28:08] "POST /bot HTTP/1.1" 200 -



MENSAJE RECIBIDO:  texto: el dólar cerró las operaciones interbancarias en $4.904, dato de cierre más alto en la historia, lo que representó un aumento de $64 frente al cierre anterior.
de acuerdo con los sistemas de información cambiaria, el precio mínimo del dólar fue de $4.840 y el máximo llegó a los $4.921.



el precio promedio de la divisa fue de $4.885, dato que repunta $70 frente a la trm.
con la trm de $4.885,50 vigente para este viernes, la devaluación del peso frente al dólar este año llega a 22,71%, mientras que el indicador en los últimos meses es de 29,50%.

ayer el ministro de hacienda, josé antonio ocampo dijo en el marco del gran foro: reforma tributaria en la recta final, organizado por el tiempo, portafolio y probogotá que “obviamente al ministerio de hacienda le preocupa, pero creo que ha habido una sobrerreacción de los mercados”.

“quiero ser muy enfático, como lo he sido todos los días, de que el gobierno tiene un compromiso con la estabilidad macroeconómica y, 

INFO:werkzeug:127.0.0.1 - - [22/Oct/2022 03:28:31] "POST /bot HTTP/1.1" 200 -
